In [1]:
%matplotlib inline
import pathlib

import datashader as ds
import fiona.crs
import geopandas as gpd
import holoviews as hv
import xarray as xr

from colorcet import fire
from datashader import transfer_functions as tf
from holoviews import opts
from shapely.geometry import box
import numpy as np
import hvplot.xarray  # noqa

hv.extension("matplotlib")
hv.output(engine="matplotlib")


if (cached := pathlib.Path("data/2021.tif")).exists():
    data = xr.open_dataset(cached, engine="rasterio").sel(dict(band=1))
else:
    data = (
        xr.open_dataset(
            "/Volumes/TOSHIBA/nighttime_light/annual/v20/2021/"
            "VNL_v2_npp_2021_global_vcmslcfg_c202203152300.average_masked.tif",
            engine="rasterio"
        )
        .loc[dict(band=1, x=slice(-20, 60), y=slice(-40, 40, -1))]
        .fillna(0).clip(0)
    )
    data.rio.to_raster(cached)
gdf = gpd.read_file("data/geometry.json", driver="GeoJSON")


bounds = gdf.query("NAME in ['Uganda', 'Tanzania', 'Mozambique', 'Kenya']").bounds
minx = bounds["minx"].min()
miny = bounds["miny"].min()
maxx = bounds["maxx"].max()
maxy = bounds["maxy"].max()
padding = 2
roi_data = data.sel(x=slice(minx - padding, maxx + padding), y=slice(miny - padding, maxy + padding))

In [2]:
opts.defaults(opts.Layout(sublabel_format='', fig_size=150))
dpi = 1000


def yformatter(value):
    return f"{abs(value):.0f}°{'N' if value > 0 else '' if value == 0 else 'S'}"


def xformatter(value):
    if int(value) == value:
        deg = f"{abs(value):.0f}"
    else:
        deg = f"{abs(value):.1f}"
    return f"{deg}°{'N' if value > 0 else '' if value == 0 else 'S'}"


africa_image = tf.shade(ds.Canvas().raster(data, agg=ds.mean(column="band_data")), cmap=fire)
africa_image = hv.RGB(np.array(africa_image.to_pil()), bounds=(-20, -40, 60, 40)).opts(
    aspect=1,
    xlabel="",
    ylabel="",
    xformatter=xformatter,
    yformatter=yformatter,
)

aspect = len(roi_data.coords["x"]) / len(roi_data.coords["y"])
roi_image = tf.shade(ds.Canvas().raster(roi_data, agg=ds.mean(column="band_data")), cmap=fire)
roi_image = hv.RGB(np.array(roi_image.to_pil()), bounds=(minx - padding, miny - padding, maxx + padding, maxy + padding)).opts(
    aspect=aspect,
    xlabel="",
    ylabel="",
    xformatter=xformatter,
    yformatter=yformatter,
)
image = (africa_image + roi_image).opts(aspect_weight=True, tight=True, shared_axes=False)
fig = hv.render(image)
fig.set_dpi(dpi)
ax1, ax2 = fig.axes
ax2.set_xticks(np.arange(30, 50, 10))
ax2.set_yticks(np.arange(-20, 10, 10))
ax2.tick_params(labelbottom=True, labeltop=True, labelleft=True, labelright=True, bottom=True, top=True, left=True, right=True)
ax2.grid()
ax1.set_xticks(np.arange(-20, 80, 20))
ax1.set_yticks(np.arange(-40, 60, 20))
ax1.tick_params(labelbottom=True, labeltop=True, labelleft=True, labelright=True, bottom=True, top=True, left=True, right=True)
ax1.grid()

bbox = box(minx - padding, miny - padding, maxx + padding, maxy + padding)
geo = gpd.GeoDataFrame({"geometry": bbox}, index=[0], crs=fiona.crs.from_epsg(4326))
geo.plot(color="blue", alpha=0.5, ax=ax1)

new_gdf = gdf.query("NAME in ['Uganda', 'Tanzania', 'Mozambique', 'Kenya']").copy()
new_gdf["coord"] = new_gdf['geometry'].apply(lambda x: x.representative_point().coords[0])
new_gdf.plot(facecolor="none", edgecolor="grey", ax=ax2)
# annotate country names
for i, row in new_gdf.iterrows():
    x, y = row["coord"]
    name = row["NAME"]
    rotation = 0
    if name == "Mozambique":
        rotation = 40
        y -= 1
        x += 2
    ax2.text(
        x, y, s=name,
        horizontalalignment='center',
        color="white",
        rotation=rotation,
    )
ax1.text(-19, -39, "CRS 4326\nSource: Annual VNL V2\nResolution: 15 arc-seconds", color="white")
fig.savefig("research_area.jpg", transparent=True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
